In [1]:
import dask.dataframe as dd
from keras.models import Sequential
from keras.layers import LSTM, Dense
from scikeras.wrappers import KerasClassifier
from dask_ml.wrappers import Incremental
from dask_ml.model_selection import train_test_split
from dask_ml.preprocessing import StandardScaler

In [2]:
data = dd.read_csv('../data/intervehicle.csv')

In [3]:
y_data = data.pop('Label')
x_data = data

In [4]:
train_x,test_x,train_y,test_y = train_test_split(x_data,y_data,test_size=0.2,random_state=0,shuffle=True)

In [5]:
train_x = train_x.reset_index(drop=True)
test_x = test_x.reset_index(drop=True) 
train_y = train_y.reset_index(drop=True)
test_y = test_y.reset_index(drop=True)

In [6]:
scaler = StandardScaler()
train_x = scaler.fit_transform(train_x)
test_x = scaler.transform(test_x)

In [8]:
model = Sequential()
model.add(LSTM(100,input_shape=(len(x_data.columns),1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100)               40800     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 40,901
Trainable params: 40,901
Non-trainable params: 0
_________________________________________________________________


In [9]:
classifier = KerasClassifier(model=model, random_state=0, optimizer='adam', epochs=10)

In [10]:
inc = Incremental(classifier,scoring='accuracy',random_state=0)

In [ ]:
inc.fit(train_x,train_y,classes=['ddos','Benign'])

In [ ]:
inc.score(test_x,test_y)

0.9836426197688439